<a href="https://colab.research.google.com/github/darkwingpatil/Ml_hackethons/blob/main/Hackethon_U2_MH1_AuthorIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Problem Statement

The problem is to identify the author of a  book from a given list of possible authors.

## Learning Objectives

At the end of the experiment, you will be able to:

* Use NLTK package
* Extract handcrafted features
* Preprocess the text
* Write an algorithm to identify the author of a given book


In [ ]:
#@title  Mini Hackathon Walkthrough
from IPython.display import HTML

HTML("""<video width="854" height="480" controls>
  <source src="https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Walkthrough/authoridentification.mp4" type="video/mp4">
</video>
""")

## Background

Author identification is the task of identifying the author of a given text. It can be considered as a typical classification problem, where a set of books with known authors are used for training. The aim is to automatically determine the corresponding author of an anonymous text.

## Grading = 10 Marks

## Setup Steps

In [66]:
#@title Run this cell to complete the setup for this Notebook

from IPython import get_ipython
ipython = get_ipython()

notebook="U2_MH1_AuthorIdentification" #name of the notebook
Answer = "This notebook is graded by mentors on the day of hackathon"
def setup():
    ipython.magic("sx wget https://cdn.talentsprint.com/talentsprint1/archives/sc/aiml/experiment_related_data/AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.rar")
    ipython.magic("sx unrar e /content/AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.rar")
    print ("Setup completed successfully")
    return

setup()

Setup completed successfully


### NOTE: You are allowed to use ML libraries such as Sklearn, NLTK, etc wherever applicable

### Downloading the required nltk Packages before moving ahead

In [1]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## **Stage 1:** Dataset Preparation

### 1 Marks -> Ensure you appropriately split the multiple short stories for the below-mentioned authors, Which will be your training data.

**1.** Before moving ahead choose two authors based on your team-number allocation: <br/>


Team=1,5,9,13,17,21  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    Author-A Vs Author-B <br />
Team=2,6,10,14,18,22 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;         Author-B Vs Author-C <br />
Team=3,7,11,15,19,23 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;         Author-C Vs Author-D <br />
Team=4,8,12,16,20,24 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;           Author-D Vs Author-E <br />



**2.** Link to the short stories collection of each author for your problem: <br />

*   Author-A -> Rudyard Kipling   [Short Stories Collection](http://www.gutenberg.org/files/2781/2781-0.txt) &nbsp;&nbsp;
*   Author-B -> Anton Chekhov [Short Stories Collection](http://www.gutenberg.org/files/1732/1732-0.txt) &nbsp;&nbsp;
*   Author-C -> Guy De Maupassant [Short Stories Collection](http://www.gutenberg.org/cache/epub/21327/pg21327.txt)&nbsp;&nbsp;
*   Author-D -> Mark Twain [Short Stories Collection](http://www.gutenberg.org/files/245/245-0.txt)&nbsp;&nbsp;
*   Author-E -> Saki [Short Stories Collection](http://www.gutenberg.org/files/1477/1477-0.txt)&nbsp;&nbsp;

**Hint for downloading raw text from Gutenberg :**  Refer to the section "Electronic Books" in the following  [link](https://www.nltk.org/book/ch03.html) for the instructions.



**Hint for finding the index of a text:**   You may use `raw.find()` and `raw.rfind()` in the same [link](https://www.nltk.org/book/ch03.html) to find the appropriate index of the start and end location

**Hint for splitting the multiple stories:** Split the stories using long space (white space character)

**Note:** Ignore the table of contents section from the given stories

In [2]:
# YOUR CODE HERE for downloading and splitting the multiple stories of respective authors which are allocated to you
import pandas as pd
import requests
from urllib import request

def getData(url):
  response = request.urlopen(url)
  return response

auth_1= getData("https://www.gutenberg.org/files/2781/2781-0.txt").read().decode('utf8')
auth_2= getData("https://www.gutenberg.org/files/1732/1732-0.txt").read().decode('utf8')

def no_of_stories(auth,index):

  extract= auth[index:2000].split('\r\n\r\n\r\n\r\n\r\n')

  outi=extract[0].split('\r\n\r\n')
  innerOuti= outi[1].split('\n')
  # print(outi)
  return len(innerOuti)


auth_1_stories=no_of_stories(auth_1,auth_1.index('CONTENTS'))

auth_2_stories=no_of_stories(auth_2,auth_2.index('CONTENTS'))

print(auth_1_stories,auth_2_stories)



12 21


In [3]:
def get_stor_details(words):

  data=words.split("\r\n\r\n\r\n\r\n\r\n")
  auth_stories_mapper={}
  for sentence in data:
    if(len(sentence) > 1000):
      heading = sentence.split('\n')
      auth_stories_mapper[heading[0]]=sentence

  return auth_stories_mapper


auth_1_map= get_stor_details(auth_1)
auth_2_map= get_stor_details(auth_2)



In [4]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
import string
from nltk import pos_tag
stopwords = nltk.corpus.stopwords.words('english')

def unique_words(words):
  split_words = words.split(" ")
  unique_words = set(word.lower() for word in split_words if word not in stopwords and word.isalnum() )
  return len(unique_words)

def avg_sent_length(words):
    sentences = sent_tokenize(words)
    return sum(len(word_tokenize(sentence)) for sentence in sentences) / len(sentences)


def lexical_richness(words):
    return len(set(words)) / len(words)

def avg_word_length(words):
    return sum(len(word) for word in words) / len(words)

def punctuation_density(text):
    punctuation_count = sum(1 for char in text if char in string.punctuation)
    word_count = len(word_tokenize(text))
    return punctuation_count / word_count * 100





## **Stage 2**: Experiment with Handcrafted features representation
Extract Handcrafted features for the obtained short stories from **Stage-1**

**Stylometry:**

Each person has a unique vocabulary, sometimes rich, sometimes limited. Although a larger vocabulary is usually associated with literary quality, this is not always the case. Ernest Hemingway is famous for using a surprisingly small number of different words in his writing, which did not prevent him from winning the Nobel Prize for Literature in 1954.

Some people write in short sentences, while others prefer long blocks of text consisting of many clauses. No two people use semicolons, em-dashes, and other forms of punctuation in the same way.




**You may explore the following ways to analyze the text and generate handcrafted features by searching text in a probing way:**

a)  Could the style of punctuation usage help as a handcrafted feature? Both by those who follow punctuations and by those who don't? Interesting [link](https://qwiklit.com/2014/03/05/top-10-authors-who-ignored-the-basic-rules-of-punctuation/)

b) The same word can sometimes be used in different contexts repeatedly by different authors. Could this fact be converted as a handcrafted feature? [link](https://www.nltk.org/book/ch01.html)

c) The above two are merely examples; As you might have noticed already the NLTK book [link](https://www.nltk.org/book/) offers several methods of analyzing and understanding the text. Each of these analyses is in itself capable of being a handcrafted feature. **However for your evaluation a minimal set of useful handcrafted features which is helping you prove a classification of an is sufficient**

d) Could most common words be used to distinguish authors?  Refer "Counting Vocabulary" section of the [link](https://www.nltk.org/book/ch01.html)

e) How about using a count of most frequently used bi-gram, tri-grams, and using it to classify an author?

f) How about using the frequency histogram of the most frequently used words across the stories by a given author a useful feature?

The limit here is endlessly limited only by your imagination, and of course your accuracy! :)


### 1 Marks ->  a) List 6 handcrafted features to distinguish author stories.

In [5]:
# For eg:
# 1. UniqueWords
# 2. AvgSentLength
# List the other handcrafted features here

def generate_dataFrame(auth1_mapper,author):
  list_features=[]
  for index,data in enumerate(auth1_mapper):
    uniq_word= unique_words(auth1_mapper[data])
    avg_sen= avg_sent_length(auth1_mapper[data])
    lex_rich=lexical_richness(auth1_mapper[data])
    avg_word_len=avg_word_length(auth1_mapper[data])
    punctuation_dens=punctuation_density(auth1_mapper[data])

    # print(data)
    # print(uniq_word,'uniq_word')
    # print(avg_sen,'avg_sen')
    # print(lex_rich,"lex_rich")
    # print(avg_word_len,"avg_word_len")
    # print(punctuation_dens,"punctuation_dens")

    data={
        "Stories_list":index,
        "story":auth1_mapper[data],
        "uniq_word":uniq_word,
        "avg_sen":avg_sen,
        "lex_rich":lex_rich,
        "avg_word_len":avg_word_len,
        "punctuation_dens":punctuation_dens,
        "author":author

    }

    list_features.append(data)
  return list_features


  # data={
  #     "Stories_list"
  # }




###  2 Marks -> b) Write functions for any 4 of the above 6 handcrafted features and label your authors accordingly.

- Get any 4 hand crafted features from the above listed 6 hand-crafted features for every story obtained from **stage-1**.
- Identify your target variable as an author and label them accordingly.

In [6]:
# Stories_list    UniqueWords    AvgSentLength     Label
#     1               x1               x2            y

# YOUR CODE HERE
auth1_list=generate_dataFrame(auth_1_map,1)
auth2_list=generate_dataFrame(auth_2_map,2)

df_auth_1 =pd.DataFrame(auth1_list)
df_auth_2 = pd.DataFrame(auth2_list)


df = pd.concat([df_auth_1, df_auth_2], ignore_index=True)

print(df.tail())

    Stories_list                                              story  \
29            16  PANIC FEARS\r\n\r\nDURING all the years I have...   
30            17  THE BET\r\n\r\nIT WAS a dark autumn night. The...   
31            18  THE HEAD-GARDENER’S STORY\r\n\r\nA SALE of flo...   
32            19  THE BEAUTIES\r\n\r\nI\r\n\r\nI REMEMBER, when ...   
33            20  THE SHOEMAKER AND THE DEVIL\r\n\r\nIT was Chri...   

    uniq_word    avg_sen  lex_rich  avg_word_len  punctuation_dens  author  
29        459  22.260504  0.004917           1.0         13.590034       2  
30        559  23.970803  0.004255           1.0         12.302071       2  
31        438  27.329268  0.005507           1.0         12.628291       2  
32        625  38.099010  0.003160           1.0         13.799376       2  
33        499  24.174242  0.004114           1.0         13.851457       2  


##**Stage 3:** Experiment with Text processing and representation:
Extract features using TFIDF or CountVectorizer or Word2vec for the obtained short stories from **Stage-1**



### 1 Mark -> a) Performing basic cleanup operations such as removing the newline characters and removing trailing spaces

**For example,** Your sentence looks as follows \[' This is a sentence\n\r. Another sentence \n'].

After newline removal from the above example, your sentence will look like \['This is a sentence. Another sentence'].

 In order to do this, you can try using a combination of split() and join()

In [7]:
# YOUR CODE HERE

df['story'] = df['story'].apply(lambda x: x.replace('\n','').replace('\r',''))



In [67]:
pip install gensim


In [65]:
wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

SyntaxError: invalid syntax (<ipython-input-65-46bc1ba01cfd>, line 1)

###  2 Marks-> b) Generate vectors for the given stories

Create a representation of text, convert it into vectors (numbers)


**Use any one** of the following algorithms for this task :

* Countvectorizer or
* TFIDFVectorizer or
* Word2Vec (The word2vec bin file (AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD) can be downloaded as a part of setup  )
  * perform sentence level tokenization and word level tokenization for the given stories

    **Example of sentences as list of words:**<br/>
    **Before:** ['This is a sentence .' , ' Another sentence']<br/>
    **After:** ['This', 'is' ,'a', 'sentence' , ' . ' , ' Another ', ' sentence ' ]
 * Assign the respective label associated for each vector representation of the extracted word

References Documents:

1.   [Countvectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
2.  [TFIDFVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)


In [69]:
# YOUR CODE HERE (HINT: Convert to numpy array if needed)
#  countVectorizer
import numpy as np
import gensim
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import csr_matrix

count_vectorizor = CountVectorizer(binary=True,ngram_range=(2,3))
model = gensim.models.KeyedVectors.load_word2vec_format('AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin', binary=True,limit=500000)

word_vector = model['king']
print(word_vector)
tfidf_vectorizor = TfidfVectorizer()

stories_count_vectorized = count_vectorizor.fit_transform(df['story'])

stories_tfdif_vectorized = tfidf_vectorizor.fit_transform(df['story'])


[ 1.25976562e-01  2.97851562e-02  8.60595703e-03  1.39648438e-01
 -2.56347656e-02 -3.61328125e-02  1.11816406e-01 -1.98242188e-01
  5.12695312e-02  3.63281250e-01 -2.42187500e-01 -3.02734375e-01
 -1.77734375e-01 -2.49023438e-02 -1.67968750e-01 -1.69921875e-01
  3.46679688e-02  5.21850586e-03  4.63867188e-02  1.28906250e-01
  1.36718750e-01  1.12792969e-01  5.95703125e-02  1.36718750e-01
  1.01074219e-01 -1.76757812e-01 -2.51953125e-01  5.98144531e-02
  3.41796875e-01 -3.11279297e-02  1.04492188e-01  6.17675781e-02
  1.24511719e-01  4.00390625e-01 -3.22265625e-01  8.39843750e-02
  3.90625000e-02  5.85937500e-03  7.03125000e-02  1.72851562e-01
  1.38671875e-01 -2.31445312e-01  2.83203125e-01  1.42578125e-01
  3.41796875e-01 -2.39257812e-02 -1.09863281e-01  3.32031250e-02
 -5.46875000e-02  1.53198242e-02 -1.62109375e-01  1.58203125e-01
 -2.59765625e-01  2.01416016e-02 -1.63085938e-01  1.35803223e-03
 -1.44531250e-01 -5.68847656e-02  4.29687500e-02 -2.46582031e-02
  1.85546875e-01  4.47265

In [91]:

def gen_word2_vec():
    word_2_vec1 = []
    for story in df['story']:
        words = story.split(" ")
        word_2_vec_word = []
        for word in words:
            try:
                if word not in stopwords:
                    word_rep = model[word]
                    word_2_vec_word.append(word_rep)
            except KeyError:
                pass

        if word_2_vec_word:
            averaged_vec = np.mean(word_2_vec_word, axis=0)
        else:
            # Handle case where no valid words were found (empty story)
            averaged_vec = np.zeros_like(word_rep)  # Assuming word_rep has a defined shape

        word_2_vec1.append(averaged_vec)

    return np.array(word_2_vec1)
data=gen_word2_vec()



In [92]:
# print(df['story'][0])
# list_of_story = [df['story'][0]]
# outi=count_vectorizor.fit_transform(list_of_story)
# out = np.array(outi)
# out

df['count_vectorizer']=list(stories_count_vectorized)
df['tfdif_vectorized']=list(stories_tfdif_vectorized)
df['word2vec']=data


In [15]:
df.head()


,Stories_list,story,uniq_word,avg_sen,lex_rich,avg_word_len,punctuation_dens,author,count_vectorizer,tfdif_vectorized
0,0,"HOW THE WHALE GOT HIS THROATIN the sea, once u...",192,38.117647,0.009822,1.0,19.907407,1,"(0, 4677)\t1\n (0, 9584)\t1\n (0, 11138)\t...","(0, 7706)\t0.015098332151410945\n (0, 4105)..."
1,1,HOW THE CAMEL GOT HIS HUMPNOW this is the next...,179,38.250000,0.010769,1.0,16.412491,1,"(0, 4677)\t1\n (0, 9584)\t1\n (0, 3992)\t1...","(0, 4644)\t0.013271837295003289\n (0, 5578)..."
2,2,HOW THE RHINOCEROS GOT HIS SKINONCE upon a tim...,170,29.058824,0.012061,1.0,12.854251,1,"(0, 4677)\t1\n (0, 9584)\t1\n (0, 3992)\t1...","(0, 1712)\t0.012677139127541254\n (0, 5601)..."
3,3,HOW THE LEOPARD GOT HIS SPOTSIN the days when ...,307,29.010638,0.005278,1.0,15.474881,1,"(0, 4677)\t1\n (0, 9584)\t1\n (0, 3992)\t1...","(0, 7617)\t0.008799384335401006\n (0, 10224..."
4,4,THE ELEPHANT’S CHILDIN the High and Far-Off Ti...,346,41.435897,0.004087,1.0,17.326733,1,"(0, 4677)\t1\n (0, 9584)\t1\n (0, 3992)\t1...","(0, 11257)\t0.010104619084904782\n (0, 8388..."


In [16]:
#  tdidf

df['author'].unique()

array([1, 2])

In [12]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.9 MB/s eta 0:00:00


###  1 Mark -> c) Is stop word removal necessary in the context of author identification? Your thoughts below?

In [94]:
# YOUR ANSWER IN TEXT
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.sparse import vstack
from sklearn.metrics import accuracy_score

X_train_ct,X_test_ct,y_train_ct,y_test_ct =  train_test_split(stories_count_vectorized,df['author'],test_size=0.2,random_state=40)
X_train_tf,X_test_tf,y_train_tf,y_test_tf =  train_test_split(stories_tfdif_vectorized,df['author'],test_size=0.2,random_state=40)
X_train_w2v,X_test_w2v,y_train_w2v,y_test_w2v =  train_test_split(data,df['author'],test_size=0.2,random_state=40)

# X_train_ct = vstack(X_train_ct.tolist()).toarray()
# X_test_ct = vstack(X_test_ct.tolist()).toarray()
# X_train_tf = vstack(X_train_tf.tolist()).toarray()
# X_test_tf = vstack(X_test_tf.tolist()).toarray()
dt = DecisionTreeClassifier()

dt.fit(X_train_w2v,y_train_w2v)
predict = dt.predict(X_test_w2v)

In [27]:
accuracy_score(predict,y_test_ct)

0.7142857142857143

In [38]:


def objective(trial):
  max_depth = trial.suggest_int('max_depth',1,10)
  min_samples_leaf = trial.suggest_int('min_samples_leaf',1,10)
  clf= DecisionTreeClassifier(max_depth=max_depth,min_samples_leaf=min_samples_leaf,random_state=40)

  score = cross_val_score(clf,X_train_ct,y_train_ct,cv=4,scoring='accuracy').mean()
  return score



def objective_knn(trial):
  best_neighbours = trial.suggest_int('n_neighbors',1,len(y_train_ct)-1)
  clf = KNeighborsClassifier(n_neighbors=best_neighbours,)
  score = cross_val_score(clf,X_train_ct,y_train_ct,cv=4,scoring='accuracy').mean()
  return score



In [36]:
import optuna

def best_dec(model):
  study = optuna.create_study(direction='maximize')
  clb=objective_knn
  if model == 'DT':
    clb = objective
  study.optimize(clb,n_trials=50)
  print(study.best_params)


best_dec('DT')

# best_dec('KNN')
# {'max_depth': 4, 'min_samples_leaf': 7}
#  n-neightbour 11

[I 2024-07-10 19:08:34,998] A new study created in memory with name: no-name-8f8e1b1c-832c-4768-92be-4bfae4b01ef1
[I 2024-07-10 19:08:35,029] Trial 0 finished with value: 0.5833333333333334 and parameters: {'max_depth': 3, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.5833333333333334.
[I 2024-07-10 19:08:35,055] Trial 1 finished with value: 0.6607142857142857 and parameters: {'max_depth': 4, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.6607142857142857.
[I 2024-07-10 19:08:35,081] Trial 2 finished with value: 0.6607142857142857 and parameters: {'max_depth': 2, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.6607142857142857.
[I 2024-07-10 19:08:35,110] Trial 3 finished with value: 0.6607142857142857 and parameters: {'max_depth': 3, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.6607142857142857.
[I 2024-07-10 19:08:35,139] Trial 4 finished with value: 0.5833333333333334 and parameters: {'max_depth': 10, 'min_samples_leaf': 2}. Best is trial 1 with value: 

{'max_depth': 4, 'min_samples_leaf': 7}


In [95]:
classifer= {
    'rf' : RandomForestClassifier(random_state=40),
    'dt' : DecisionTreeClassifier(random_state=40),
    'lg': LogisticRegression(random_state=40),
    'knn': KNeighborsClassifier(n_neighbors=11),
    'svc':SVC(kernel='linear')

}

def classify_for_datasets(X_train,X_test,y_train,y_test):

  for model in classifer:
    md = classifer[model]
    md.fit(X_train,y_train)
    x_pred = md.predict(X_test)

    print(f"Accuracy by {model} is {accuracy_score(x_pred,y_test)}")


# classify_for_datasets(X_train_ct,X_test_ct,y_train_ct,y_test_ct)

# classify_for_datasets(X_train_tf,X_test_tf,y_train_tf,y_test_tf)

classify_for_datasets(X_train_w2v,X_test_w2v,y_train_w2v,y_test_w2v)

Accuracy by rf is 0.8571428571428571
Accuracy by dt is 0.7142857142857143
Accuracy by lg is 0.7142857142857143
Accuracy by knn is 0.8571428571428571
Accuracy by svc is 0.8571428571428571


##**Stage 4:** Classification :

### Expected accuracy is above 85%

### 2 Marks -> Perform a classification using either features obtained from Stage2 or Stage3

In [ ]:
# YOUR CODE HERE

# Further Ideas for exploration after the hackathon:

**Statistical analysis** of text using NLP, by analysis meaning of sentences, feature based grammars and analyzing structure of sentences!

reference: www.nltk.org/book